In [ ]:
import numpy as np
import torchvision.transforms as transforms
import torch

from src.extractor_utils import (predict, save_prediction)
from src.Dataset import KaggleSafeDriverDataset
from src.imgnet_utils import denormalize
from src.data_loader import _create_dataLoader
from src.data_loader import _create_dataLoader
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

# Choosing resources to extrac the features

In [ ]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

In [ ]:
use_gpu = True
use_DataParalel= True
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))

    else:
        print('Using only one GPU')

if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

# Path to raw data

In [ ]:
path2train = "/mnt/home/e209440/data/train" 
path2test = "/mnt/home/e209440/data/test" 

# Image transformation

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

# Creating Dataset & DataLoader

In [ ]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [ ]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}

In [ ]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [ ]:
dset_sizes = {x: len(dsets[x]) for x in ['train','valid', 'test']} 
dset_classes = len(dsets['train'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

# Checking GPU usage

In [ ]:
if use_gpu:
    !free -h
    !nvidia-smi

# Choosing Model

In [ ]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

In [ ]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model = torch.nn.DataParallel(model) #device_ids=[1,3]
        model.cuda()
        convnet = model.module.mrnc
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
        convnet = model.mrnc 

if use_CPU:
    print("Using CPU's")
    convnet = model.mrnc 

# Extracting the features

In [ ]:
#extract features from images
convOutput_train = predict(dset_loaders['train'], convnet,use_gpu=use_gpu)
convOutput_valid = predict(dset_loaders['valid'], convnet,use_gpu=use_gpu)
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=use_gpu)

In [ ]:
print(convOutput_train['true'].size(), convOutput_train['pred'].size())
print(convOutput_valid['true'].size(), convOutput_valid['pred'].size())
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

In [ ]:
print(convOutput_train['true'].type(), convOutput_train['pred'].type())

# Saving features

In [ ]:
sav_feats= {
    'train': (convOutput_train['pred'], convOutput_train['true'],model_name),
    'valid': (convOutput_valid['pred'], convOutput_valid['true'],model_name),
    'test': (convOutput_test['pred'], convOutput_test['true'],model_name)

}

In [ ]:
sav_feats['train'][2]

In [ ]:
path2features = './features/'

In [ ]:
save_prediction(path2features,sav_feats)